In [1]:
%run ../00_default_options.ipynb

In [4]:
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

In [5]:
cohort = Cohort.load(id=1)

In [6]:
onset_dataframe = OnsetDataFrame.load(id=18)

In [7]:
feature_pipeline = FeaturePipeline.load(id=19)

In [8]:
import morpher
from morpher.jobs import *

Using TensorFlow backend.


In [9]:
agg_func_regex = 'count__'

def prepare_data_function(data, target):
    import morpher
    from morpher.jobs import Impute, Scale
    from sklearn.preprocessing import OneHotEncoder
    from enum import Enum
    
    # remove unused features
    del data['medical_record_number']
    del data['mother_account_number']
    del data['date_of_birth']
    del data['month_of_birth']
    del data['patient_ethnic_group']
    del data['religion']
    del data['address_zip']
    del data['deceased_indicator']
    del data['marital_status_code']
    
    class RaceType(str, Enum):
        AFRICAN = 'African'
        AMERICAN_BLACK = 'Black or African-American'
        AMERICAN_NATIVE = 'Native American'
        ASIAN = 'Asian'
        ASIAN_PACIFIC = 'Asian Pacific'
        ASIAN_INDIAN = 'Asian Indian'
        ASIAN_CHINESE = 'Asian Chinese'
        HISPANIC = 'Hispanic or Latino'
        OTHER = 'Other'
        WHITE = 'White'
    
    RACE_MAPPING = {
        RaceType.AFRICAN: [
            'Cape Verdian',
            'Congolese',
            'Eritrean',
            'Ethiopian',
            'Gabonian',
            'Ghanaian',
            'Guinean',
            'Ivory Coastian',
            'Kenyan',
            'Liberian',
            'Madagascar',
            'Malian',
            'Nigerian',
            'Other: East African',
            'Other: North African',
            'Other: South African',
            'Other: West African',
            'Senegalese',
            'Sierra Leonean',
            'Somalian',
            'Sudanese',
            'Tanzanian',
            'Togolese',
            'Ugandan',
            'Zimbabwean'
        ],
        RaceType.AMERICAN_BLACK: [
            'African American (Black)',
            'African-American',
            'Black Or African-American',
            'Black or African - American',
        ],
        RaceType.AMERICAN_NATIVE: [
            'American (Indian/Alaskan)',
            'Native American'
        ],
        RaceType.ASIAN: [
            'Asian',
            'Bangladeshi',
            'Bhutanese',
            'Burmese',
            'Cambodian',
            'Hmong',
            'Indonesian',
            'Japanese',
            'Korean',
            'Laotian',
            'Malaysian',
            'Maldivian',
            'Nepalese',
            'Okinawan',
            'Pakistani',
            'Singaporean',
            'Taiwanese',
            'Thai',
            'Vietnamese',
            'Yapese'
        ],
        RaceType.ASIAN_PACIFIC: [
            'Asian (Pacific Islander)',
            'Carolinian',
            'Chamorro',
            'Chuukese',
            'Fijian',
            'Filipino',
            'Guamanian',
            'Guamanian Or Chamorro',
            'Guamanian or Chamorro',
            'Iwo Jiman',
            'Kiribati',
            'Kosraean',
            'Mariana Islander',
            'Marshallese',
            'Melanesian',
            'Micronesian',
            'Native Hawaiian',
            'New Hebrides',
            'Other Pacific Islander',
            'Pacific Islander',
            'Palauan',
            'Pohnpeian',
            'Polynesian',
            'Saipanese',
            'Samoan',
            'Papua New Guinean',
            'Tahitian',
            'Tokelauan',
            'Tongan'
        ],
        RaceType.ASIAN_INDIAN: [
            'Asian Indian',
            'Sri Lankan',
            'Sri lankan',
            'West Indian'
        ],
        RaceType.ASIAN_CHINESE: [
            'Chinese',
        ],
        RaceType.HISPANIC: [
            'Barbadian',
            'Dominica Islander',
            'Grenadian',
            'Haitian',
            'Hispanic/Latino',
            'Jamaican',
            'St Vincentian',
            'Trinidadian'
        ],
        RaceType.OTHER: [
            '',
            'Aa',
            'Ab',
            'Af',
            'Ag',
            'Ak',
            'Al',
            'Ap',
            'Ar',
            'Av',
            'Ay',
            'B',
            'B1',
            'B2',
            'B3',
            'B4',
            'B5',
            'B6',
            'B7',
            'B8',
            'B9',
            'Ba',
            'Bb',
            'Bc',
            'Bd',
            'Be',
            'Bf',
            'Bg',
            'Bh',
            'Bj',
            'Bk',
            'Bm',
            'Bn',
            'Bo',
            'Bp',
            'Bq',
            'Br',
            'Bs',
            'Bt',
            'Bu',
            'Bv',
            'Bw',
            'Bx',
            'By',
            'Bz',
            'I',
            'MSDW_NOT APPLICABLE',
            'MSDW_OTHER',
            'MSDW_UNKNOWN',
            'NOT AVAILABLE',
            'Non Hispanic',
            'O',
            'Other',
            'Pk',
            'Pl',
            'Pm',
            'Po',
            'Ps',
            'Pv',
            'U',
            'Unk',
            'Unknown',
            'W'
        ],
        RaceType.WHITE: [
            'Caucasian (White)',
            'White'
        ]
    }
    
    # value mapping
    data['race'] = (
        data.race.map({
            label: cat for cat, labels in RACE_MAPPING.items()
            for label in labels
        }).astype(pd.api.types.CategoricalDtype(RaceType))
    )
    
    # pre-encoding nan handling
    for column in ['gender', 'race']:
        enc = OneHotEncoder(sparse=False)
        transformed_data = enc.fit_transform(data[[column]])
        transformed_columns = pd.DataFrame(
            data=transformed_data, 
            columns=[f'{column}_{c}' for c in enc.categories_[0]]
        )
        data = data.join(transformed_columns)
        del data[column]
        
    label_encoded_data = data.copy()
    
    scaled_data, _ = Scale().execute(data=label_encoded_data, target=target)
    
    imputed_data, _ = Impute().execute(data=scaled_data, imputation_method=morpher.config.imputers.DEFAULT)
    
    return imputed_data

data_loader = DataLoader(agg_func_regex, prepare_data_function)

In [10]:
from itertools import product

In [11]:
thresholds_numeric = [(x / 100) for x in range(5, 100, 5)]
observation_windows_numeric = [[x, -1] for x in range(-361, -1, 30)]
thresholds_occurring = [(x / 100) for x in range(5, 100, 5)]
observation_windows_occurring = [[x, -1] for x in range(-361, -1, 30)]

In [12]:
import morpher.config

In [13]:
iterator = product(
    thresholds_numeric,
    observation_windows_numeric,
    thresholds_occurring,
    observation_windows_occurring
)

configs = [x for x in iterator if x[0] == x[2] and x[1] == x[3]]
targets = ['other_viral_infection_onset_from_0_days_after_to_365_days_after']
algorithms = [morpher.config.algorithms.GBDT, morpher.config.algorithms.RF, morpher.config.algorithms.DT, morpher.config.algorithms.LR]
samplers = samplers = [morpher.config.samplers.RANDOM, morpher.config.samplers.URANDOM, morpher.config.samplers.SMOTE, morpher.config.samplers.NOSAMPLER]

In [ ]:
pipeline = training.execute(
    comment='baseline-2 OVI with RFE #2',
    version='32.0.0',
    cohort=cohort,
    onset_dataframe=onset_dataframe,
    feature_pipeline=feature_pipeline,
    data_loader=data_loader,
    observation_iterator=configs,
    targets=targets,
    algorithms=algorithms,
    samplers=samplers,
    feature_type_occurring="occurring",
    feature_type_numeric="numeric",
    rfe__run=True,
    rfe__step_size=50
)

Columns (11,18,19,20,24,35,64,83,90,92,94,96,97,106,114,117,120,125,126,133,136,140,141,142,143,144,150,178,235,244,248,288,311,317,320,325,326,330,334,341,353,359,360,361,365,402,421,425,430,434,436,439,446) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f6f5f9b16d8>
Fetching data for Patient (...)
Fitting estimator with 932 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 400.044ms

Fitting estimator with 882 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 27.958ms

Fitting estimator with 832 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 818.555ms

Fitting estimator with 782 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 547.823ms

Fitting estimator with 732 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 236.141ms

Fitting estimator with 682 features.
*** Training of model 'GradientBoostingClassifier' started.
*** Training of class

*** Training of classifier ready. Time elapsed: 293.237ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 362.681ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 273.82ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 241.014ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 295.445ms

Fitting estimator with 932 features.
*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 92.916ms

Fitting estimator with 882 features.
*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 82.338ms

Fitting estimator with 832 features.
*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 76.841ms

Fitting estimator with 782 fe

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 29.826ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 34.317ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 47.872ms



Columns (11,18,19,20,24,27,35,64,83,90,92,94,95,96,98,105,112,113,115,118,122,123,130,133,137,138,139,140,141,147,168,169,175,232,241,245,246,285,308,313,314,316,321,325,329,336,348,354,355,356,360,376,391,396,414,418,423,427,429,432,439) have mixed types. Specify dtype option on import or set low_memory=False.


...

Fetching data for <fiber.condition.mrns.MRNs object at 0x7f6f5ed4c940>
Fetching data for Patient (...)
*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 521.578ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 528.595ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 628.179ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 621.032ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 434.984ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 306.514ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 300.344ms

*** Training of model 'GradientBoostingClassifier' started.
*** 

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 63.075ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 56.386ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 50.054ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 19.849ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 18.462ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 26.336ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 25.066ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 84.383ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 49.566ms

*** Traini

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 6.02ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 5.78ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 5.753ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 4.716ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 4.483ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 17.071ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 8.743ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 7.165ms

*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapse

*** Training of classifier ready. Time elapsed: 656.515ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 489.146ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 495.608ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 4.419ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 4.129ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 4.903ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 5.346ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 4.633ms

*** Training of model 'DecisionTreeClassifier' started.
*** Training of classifier ready. Time elapsed: 5.073ms

*** Training of model 'DecisionTr

*** Training of classifier ready. Time elapsed: 520.091ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 548.403ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 554.095ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 549.706ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 545.128ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 324.244ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 324.435ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 329.731ms

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 325.339ms

*** Training of model

In [ ]:
pipeline.id